In [4]:
import sys, importlib
from pathlib import Path

PROJECT_ROOT = Path("/Users/sajanshergill/Machine-Learning-Projects/Trust-Graph")
sys.path.insert(0, str(PROJECT_ROOT))

importlib.invalidate_caches()

import src
print("src loaded from:", src.__file__)
print("src folder files:", [p.name for p in Path(src.__file__).parent.glob("*.py")])

src loaded from: /Users/sajanshergill/Machine-Learning-Projects/Trust-Graph/src/__init__.py
src folder files: ['signals.py', 'config.py', 'data_loader.py', '__init__.py', 'graph_builder.py', 'viz.py']


In [6]:
import os
print("cwd:", os.getcwd())
print("sys.path[0]:", sys.path[0])

cwd: /Users/sajanshergill/Machine-Learning-Projects/Trust-Graph/notebooks
sys.path[0]: /Users/sajanshergill/Machine-Learning-Projects/Trust-Graph


In [7]:
import importlib
import src.scoring
importlib.reload(src.scoring)

from src.scoring import build_business_risk_table

ModuleNotFoundError: No module named 'src.scoring'